In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import time
import json
import re
import numpy as np

In [2]:
bs=BeautifulSoup(open("mpiindex.html",encoding = "utf-8"),"html.parser")

In [3]:
urls=[]
allurl = bs.find_all("a",attrs={"data-bi-name": "tree-leaf"})
for url in allurl:
    print(url['href'])
    # 爬取英文页面
    url = url['href']#.replace("zh-cn","en-us")
    print(url)
    urls.append(url)

https://learn.microsoft.com/zh-cn/message-passing-interface/microsoft-mpi
https://learn.microsoft.com/zh-cn/message-passing-interface/microsoft-mpi
https://learn.microsoft.com/zh-cn/message-passing-interface/microsoft-mpi-release-notes
https://learn.microsoft.com/zh-cn/message-passing-interface/microsoft-mpi-release-notes
https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-reference
https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-reference
https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-enumerations
https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-enumerations
https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-comm-enumeration
https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-comm-enumeration
https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-op-enumeration
https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-op-enumeration
https://learn.microsoft.com/zh-cn/message-passing-inte

In [6]:
AllResult  =[]
Allfunction=[]
description=[]
parameters =[]
headers= {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.78'}
for i in range(5,len(urls)):#
    try:
        result = requests.get(urls[i],headers=headers)
    except:
        print("网络连接有问题")
        continue
    if result.status_code == 200:
        result.encoding="utf-8"
        newbs=BeautifulSoup(result.text,"html.parser")
        # 找到fortran代码
        outdata = newbs.find_all("code",attrs={"class": "lang-FORTRAN"})
        if len(outdata)==0 or len(outdata)>1:
            print("没有代码或有问题",urls[i])
            continue
        AllResult.append(outdata[0].text)
        # 找到函数名
        Allfunction.append(urls[i].split("/")[-1])
        # 找到函数功能描述
        dom = etree.HTML(str(newbs))
        description.append(dom.xpath('//*[@id="main"]/div[3]/p[1]')[0].text)
        # 找到参数描述
        outstr=""
        paraList = dom.xpath('//*[@id="main"]/div[3]/ul/li')
        for j in range(0,len(paraList)):
            # 解析页面上li标签中存储的参数说明
            paraDescription=dom.xpath('//*[@id="main"]/div[3]/ul/li')[j].xpath('string(.)')
            outstr+=paraDescription
        parameters.append(outstr)
        print(f"{i}/{len(urls)}", urls[i].split("/")[-1])
        print(dom.xpath('//*[@id="main"]/div[3]/p[1]')[0].text)
        time.sleep(0.5)

没有代码或有问题 https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-op-enumeration
没有代码或有问题 https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-datatype-enumeration
没有代码或有问题 https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-error
没有代码或有问题 https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-functions
没有代码或有问题 https://learn.microsoft.com/zh-cn/message-passing-interface/mpi-caching-functions
10/359 mpi-comm-create-keyval-function
创建新的属性键。
11/359 mpi-comm-delete-attr-function
删除与通信器上的键关联的属性值。
12/359 mpi-comm-free-keyval-function
释放一个已灭绝的属性键。
13/359 mpi-comm-get-attr-function
按键检索属性值。
14/359 mpi-comm-set-attr-function
存储与键关联的属性值。
15/359 mpi-comm-copy-attr-function-function
None
16/359 mpi-comm-delete-attr-function-function
None
17/359 mpi-type-copy-attr-function-callback-function
None
18/359 mpi-type-delete-attr-function-callback-function
None
19/359 mpi-win-copy-attr-function-callback-function
None
20/359 mpi-win-create-keyval-function
为 MPI 窗口对象创

In [14]:
# 结果保存
#AllResult  
#Allfunction
#description
#parameters 
AllResultOut = np.array(AllResult)
AllfunctionOut = np.array(Allfunction)
descriptionOut = np.array(description)
parametersOut = np.array(parameters)
np.save('AllResultOut',AllResultOut) 
np.save('Allfunction',Allfunction) 
np.save('description',description) 
np.save('parameters',parameters) 

In [20]:
AllResult[0]

'    MPI_COMM_CREATE_KEYVAL(COMM_COPY_ATTR_FN, COMM_DELETE_ATTR_FN, COMM_KEYVAL,\n            EXTRA_STATE, IERROR)\n        EXTERNAL COMM_COPY_ATTR_FN, COMM_DELETE_ATTR_FN\n        INTEGER COMM_KEYVAL, IERROR\n        INTEGER(KIND=MPI_ADDRESS_KIND) EXTRA_STATE\n'

In [23]:
Paralist

'INTEGER COMM, COMM_KEYVAL, IERROR\n'

In [25]:
outbody.split("(")[0]

'MPI_COMM_DELETE_ATTR'

In [22]:
outbody

'MPI_COMM_DELETE_ATTR(COMM, COMM_KEYVAL, IERROR)'

In [30]:
data={}
# 解析函数和函数参数
for i in range(0,len(AllResult)):#len(AllResult)
    Paralist=""
    # 处理换行
    each = re.sub(r",\n\s+", ", ",AllResult[i])
    # 输出函数
    outbody = each.split("\n")[0].strip()
    # 输出函数参数
    for j in range(1,len(each.split("\n"))):
        if j != len(each.split("\n"))-1:
            Paralist += each.split("\n")[j].strip()+"\n"
        else:
            Paralist += each.split("\n")[j].strip()
# 解析函数用途说明
    if description[i] != None:
        EachDescription = description[i]
    else:
        EachDescription = "None"

    data[outbody.split("(")[0]]={
        "scope":"FortranFreeForm",
        "prefix":[outbody.split("(")[0]],
        "description":outbody+"\n"+EachDescription+"\n"+Paralist+"\n"+parameters[i],
        "body":[outbody]
    }
    print(i)
json_str = json.dumps(data,ensure_ascii=False,indent=1)
with open('fortran-mpi-en.json', 'w',newline='\n') as fp:
    fp.write(json_str)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [21]:
data={}
# 解析函数和函数参数
for i in range(0,2):#len(AllResult)
    Paralist=""
    # 处理换行
    each = re.sub(r",\n\s+", ", ",AllResult[i])
    # 输出函数
    outbody = each.split("\n")[0].strip()
    # 输出函数参数
    for j in range(1,len(each.split("\n"))):
        if j != len(each.split("\n"))-1:
            Paralist += each.split("\n")[j].strip()+"\n"
        else:
            Paralist += each.split("\n")[j].strip()
# 解析函数用途说明
    if description[i] != None:
        EachDescription = description[i]
    else:
        EachDescription = "None"

    data[Allfunction[i]]={
        "scope":"FortranFreeForm",
        "prefix":[Allfunction[i]],
        "description":outbody+"\n"+EachDescription+"\n"+Paralist+"\n"+parameters[i],
        "body":[outbody]
    }
    print(i)
json_str = json.dumps(data,ensure_ascii=False,indent=1)
with open('fortran-mpi-en.json', 'w',newline='\n') as fp:
    fp.write(json_str)


0
1


In [1]:
datatype= "MPI_DATATYPE_NULL MPI_CHAR MPI_UNSIGNED_CHAR MPI_SHORT MPI_UNSIGNED_SHORT MPI_INT MPI_UNSIGNED MPI_LONG MPI_UNSIGNED_LONG MPI_LONG_LONG_INT MPI_LONG_LONG MPI_FLOAT MPI_DOUBLE MPI_LONG_DOUBLE MPI_BYTE MPI_WCHAR MPI_PACKED MPI_LB MPI_UB MPI_C_COMPLEX MPI_C_FLOAT_COMPLEX MPI_C_DOUBLE_COMPLEX MPI_C_LONG_DOUBLE_COMPLEX MPI_2INT MPI_C_BOOL MPI_SIGNED_CHAR MPI_UNSIGNED_LONG_LONG MPI_CHARACTER MPI_INTEGER MPI_REAL MPI_LOGICAL MPI_COMPLEX MPI_DOUBLE_PRECISION MPI_2INTEGER MPI_2REAL MPI_DOUBLE_COMPLEX MPI_2DOUBLE_PRECISION MPI_2COMPLEX MPI_2DOUBLE_COMPLEX MPI_REAL2 MPI_REAL4 MPI_COMPLEX8 MPI_REAL8 MPI_COMPLEX16 MPI_REAL16 MPI_COMPLEX32 MPI_INTEGER1 MPI_COMPLEX4 MPI_INTEGER2 MPI_INTEGER4 MPI_INTEGER8 MPI_INTEGER16 MPI_INT8_T MPI_INT16_T MPI_INT32_T MPI_INT64_T MPI_UINT8_T MPI_UINT16_T MPI_UINT32_T MPI_UINT64_T MPI_AINT MPI_OFFSET MPI_FLOAT_INT MPI_DOUBLE_INT MPI_LONG_INT MPI_SHORT_INT MPI_LONG_DOUBLE_INT"

In [4]:
datatypes = datatype.split(' ')

In [6]:
data={}
for datatype in datatypes:
    data[datatype]={
        "scope":"FortranFreeForm",
        "prefix":[datatype],
        "description":"MPI type "+datatype,
        "body":[datatype]
    }
json_str = json.dumps(data,ensure_ascii=False,indent=1)
with open('fortran-mpi-type.json', 'w',newline='\n') as fp:
    fp.write(json_str)

In [6]:
datatype="MPI_COMM_NULL MPI_COMM_WORLD MPI_COMM_SELF MPI_OP_NULL MPI_MAX MPI_MIN MPI_SUM MPI_PROD MPI_LAND MPI_BAND MPI_LOR MPI_BOR MPI_LXOR MPI_BXOR MPI_MINLOC MPI_MAXLOC MPI_REPLACE"

In [5]:
data={}
datatypes = datatype.split(' ')
for datatype in datatypes:
    data[datatype]={
        "scope":"FortranFreeForm",
        "prefix":[datatype],
        "description":"MPI type "+datatype,
        "body":[datatype]
    }
json_str = json.dumps(data,ensure_ascii=False,indent=1)
with open('fortran-mpi-reduce.json', 'w',newline='\n') as fp:
    fp.write(json_str)